 Create Coversational Agent
 -----------------------------
1.   Choose Topic
2.   create a prompt and create a chain that uses the prompt
3.   Create an agent and add conversational memory
4.   Add additional context to the agent
5.   Add a few-shot example to the agent







In [32]:
!pip install langchain  langchain_community langchain-openai langchain_neo4j  -q

In [33]:
import os
from langchain_openai import ChatOpenAI
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain.prompts import PromptTemplate

from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_neo4j import Neo4jChatMessageHistory, Neo4jGraph, Neo4jVector
from uuid import uuid4

In [55]:
def load_properties(path):
    data = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.split("=", 1)
                data[k.strip()] = v.strip()
    return data

In [56]:
import os
props = load_properties("openai_key.txt")

os.environ["OPENAI_KEY"] = props["OPENAI_KEY"]
os.environ["NEO4J_URI"] = props["NEO4J_URI"]
os.environ["NEO4J_USERNAME"] = props["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"] = props["NEO4J_PASSWORD"]
#os.environ["NEO4J_DATABASE"] = props["NEO4J_DATABASE"]

In [57]:
#Creating Prompt - Few-shot examples
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples:

Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Schema: {schema}
Question: {question}
"""

In [58]:
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [59]:


llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_KEY"),
    temperature=0,
    model="gpt-3.5-turbo"

)

In [60]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

In [61]:
#create chain that uses the prompt
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [62]:
tools = [
    Tool.from_function(
        name="Movie Plot Search",
        description="For when you need to compare a plot to a movie. The question will be a string. Return a string.",
        func=cypher_chain.invoke,
    ),
]


In [63]:
#Creating Agent
agent_prompt = hub.pull("hwchase17/react-chat")

custom_prompt = agent_prompt.partial(
    additional_context="""
                      You are a Neo4j Movie QA Assistant.
                      Your job is to answer questions using the graph.
                      Graph schema:
                      Movie(title, plot).
                      Always give short and accurate answers.
                      """
        )

#print(agent_prompt.template)
agent = create_react_agent(llm, tools, custom_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,handle_parsing_errors=True)

In [64]:
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: e2e8f9a4-370a-4702-9cad-8a15ee786c9d


In [65]:
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [66]:
# adding conversational memory
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [67]:
while (q := input("> ")) != "exit":

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )

    print(response["output"])

> details about Tom Hanks


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})
RETURN p.dateofbirth, p.birthcity
Full Context:
[{'p.dateofbirth': '1956-07-09', 'p.birthcity': 'Concord, California, USA'}]

> Finished chain.
Tom Hanks was born on July 9, 1956, in Concord, California, USA.
> exit
